# M4 | Research Investigation Notebook

In this notebook, you will do a research investigation of your chosen dataset in teams. You will begin by formally selecting your research question (task 0), then processing your data (task 1), creating a predictive model (task 2), evaluating your model's results (task 3), and describing the contributions of each team member (task 4).

For grading, please make sure your notebook has all cells run and is stored in your team's [Github Classroom repository](https://classroom.github.com/a/CNxME27U). You will also need to write a short, 2 page report about your design decisions as a team, to be stored in your repository. The Milestone 4 submission will be the contents of your repository at the due date (April 28 at 23:59 CET).

## Brief overview of Calcularis
[Calcularis](https://school.alemira.com/de/calcularis/) by Alemira School is a mathematics learning program developed with neuroscientists and computer scientists from ETH Zurich. It promotes the development and interaction of the different areas of the brain that are responsible for processing numbers and quantities and solving mathematical tasks. Calcularis can be used from 1st grade to high school. Children with dyscalculia also benefit in the long term and overcome their arithmetic weakness.

The Calcularis dataset has three main tables:
* ***users***: meta information about users (i.e. total time spent learning with Calcularis, geographic location).
* ***events***: events done by the users in the platform (i.e. playing a game, selecting a new animal in the zoo simulation).
* ***subtasks***: sub-tasks with answer attempts solved by users, primarily in the context of game events.

These tables and useful metadata information are described in detail in the [Milestone 2 data exploration notebook](https://github.com/epfl-ml4ed/mlbd-2023/blob/main/project/milestone-02/m2_calcularis_sciper.ipynb).

We have provided access to the [full dataset](https://moodle.epfl.ch/mod/forum/discuss.php?d=88179) (~65k users) and a randomly selected subset (~1k users from M2). We have also provided access to a [test account to experiment with Calcularis](https://moodle.epfl.ch/mod/forum/discuss.php?d=88094). You should provide arguments and justifications for all of your design decisions throughout this investigation. You can use your M3 responses as the basis for this discussion.

In [1]:
# Import the tables of the data set as dataframes.
import time
start = time.time()
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt

DATA_DIR = './../data' # You many change the directory

# You can use the nrows=X argument in pd.read_csv to truncate your data
users_small = pd.read_csv('{}/calcularis_small_users.csv'.format(DATA_DIR), index_col=0)
events_small = pd.read_csv('{}/calcularis_small_events.csv'.format(DATA_DIR), index_col=0)
subtasks_small = pd.read_csv('{}/calcularis_small_subtasks.csv'.format(DATA_DIR), index_col=0)
users_full = pd.read_csv(f'{DATA_DIR}/full_calcularis_users.csv', index_col=0)
events_full = pd.read_csv(f'{DATA_DIR}/full_calcularis_events.csv', index_col=0)
subtasks_full = pd.read_csv(f'{DATA_DIR}/full_calcularis_subtasks.csv', index_col=0)

## Task 0: Research Question

**Research question:**
For this milestone we focus on detecting wheel-spinning behaviour of Calcularis Users. This is a time series analysis. We rely on features that were proven to be useful in various scientific papers which had to goal of detecting wheel-spinning on different datasets.

## Task 1: Data Preprocessing

In this section, you are asked to preprocess your data in a way that is relevant for the model. Please include 1-2 visualizations of features / data explorations that are related to your downstream prediction task.

In [2]:
#Preprocess given data frames (clean out data)

#remove event entries which have 0 subtasks
events_full = events_full[(events_full.subtasks != "[]") & ~events_full.subtasks.isna()]

#remove event/subtasks entries which have no start value, no skill_id, no correct value
subtasks_full = subtasks_full[~subtasks_full.correct.isna()]
events_full = events_full[~events_full.start.isna() & ~events_full.skill_id.isna()]

#events_full[events_full['subtasks'] == '[]'].head()
#events_full['num_subtasks'] = subtasks_full.groupby(['event_id']).size()
#print(events_full['num_subtasks'].unique())
#print(events_full[events_full.num_subtasks.isna()])

In [3]:
# Your code for data processing goes here
events = events_small

subtasks = subtasks_small
processed_df = events.copy()
processed_df.drop_duplicates(inplace=True)
processed_df = processed_df[processed_df.type == 'task']
processed_df = processed_df[['user_id', 'skill_id', 'learning_time_ms', 'start']]
processed_df = processed_df.reset_index()
processed_df['correct'] = processed_df.apply(
    lambda row: subtasks[subtasks.event_id == row.event_id].iloc[0].correct, axis=1
)



In [4]:
processed_df['po'] = processed_df.apply(
    lambda row: processed_df[(processed_df.user_id == row.user_id) & (processed_df.skill_id == row.skill_id) & (processed_df.start <= row.start)]['event_id'].count(),
    axis=1
)
processed_df

,event_id,user_id,skill_id,learning_time_ms,start,correct,po
0,0,1,1.0,8835.0,2022-11-02T08:39:12.355Z,True,1
1,1,1,4.0,21167.0,2022-11-11T10:26:27.893Z,True,1
2,2,1,7.0,11182.0,2022-11-18T10:34:01.044Z,True,1
3,3,1,19.0,6823.0,2022-11-25T10:32:43.428Z,False,1
4,4,1,7.0,9107.0,2022-12-02T10:44:40.555Z,True,2
...,...,...,...,...,...,...,...
33654,37414,998,110.0,7635.0,2020-12-02T11:57:00.179Z,True,1
33655,37415,998,111.0,7762.0,2021-01-06T14:14:36.824Z,True,2
33656,37417,1000,1.0,9514.0,2019-09-30T10:04:31.264Z,False,1
33657,37418,1000,3.0,96077.0,2020-01-20T10:02:02.957Z,True,1


In [5]:
# Does not include result from the current practice opportunity
processed_df['correct_response_count'] = processed_df.apply(
    lambda row: processed_df[(processed_df.user_id == row.user_id) & (processed_df.skill_id == row.skill_id) & (processed_df.start < row.start)]['correct'].sum(),
    axis=1
)
processed_df

KeyboardInterrupt: 

In [ ]:
processed_df['correct_response_percentage'] = processed_df.apply(
    lambda row: row.correct_response_count / (row.po - 1) if row.po > 1 else 0,
    axis=1
)
processed_df

In [ ]:
processed_df[(processed_df.user_id == 41) & (processed_df.skill_id == 95)]

In [ ]:
processed_df = processed_df.sort_values(by='po')
for index, row in processed_df.iterrows():
    if row.po == 1:
        processed_df.loc[index, 'correct_response_in_a_row_count'] = 0
    else:
        
        last_response = processed_df[(
            processed_df.user_id == row.user_id) & (processed_df.skill_id == row.skill_id) & (
            processed_df.po == row.po-1
        )]
        processed_df.loc[index, 'correct_response_in_a_row_count'] = last_response.correct_response_in_a_row_count.values[0] + 1 if last_response.correct.values[0] else 0

processed_df

In [ ]:
processed_df['correct_response_in_a_row_percentage'] = processed_df.apply(
    lambda row: row.correct_response_in_a_row_count / (row.po - 1) if row.po > 1 else 0,
    axis=1
)
processed_df

In [ ]:
processed_df['time_on_current_skill_ms'] = processed_df.apply(
    lambda row: processed_df[
        (processed_df.user_id == row.user_id) &
        (processed_df.skill_id == row.skill_id) & 
        (processed_df.start <= row.start)
    ]['learning_time_ms'].sum(),
    axis=1
)
processed_df

In [ ]:
PO_CUTOFF = 10

In [ ]:
before = processed_df.copy()

In [ ]:
processed_df = before[before.po <= PO_CUTOFF]
print((len(before)-len(processed_df))/len(before))


In [ ]:
processed_df['pessimistic_wheelspinning'] = processed_df.apply(
    lambda row: len(processed_df[
        (processed_df.user_id == row.user_id) & 
        (processed_df.skill_id == row.skill_id) & 
        (processed_df.correct_response_in_a_row_count >= 3)
    ]) == 0,
    axis=1
)

processed_df

In [ ]:
users_with_sufficient_po = processed_df[processed_df.po == PO_CUTOFF].user_id.unique()
processed_df['optimistic_wheelspinning'] = processed_df.user_id.isin(users_with_sufficient_po) & processed_df.pessimistic_wheelspinning

In [ ]:
processed_df.drop(columns=[
    'event_id', 'learning_time_ms', 'start', 'correct'
], inplace=True)
print(time.time() - start)

In [ ]:
backup_df = processed_df.copy()
temp_df = processed_df[processed_df.correct_response_in_a_row_count == 3]
mastery_achieved = pd.DataFrame(temp_df.groupby(['user_id', 'skill_id'])['po'].min())

for index, row in processed_df.iterrows():
    if (
        ((row.user_id, row.skill_id) in mastery_achieved.index) and
        row.po >= mastery_achieved.loc[(row.user_id, row.skill_id)].po
    ):
        processed_df.drop(index=index, inplace=True)

processed_df

In [ ]:
backup_df[backup_df.user_id == 865]

In [ ]:
processed_df[(processed_df.user_id == 865) & (processed_df.skill_id == 48)]

In [ ]:
no_indeterminate_df = processed_df[processed_df.optimistic_wheelspinning == processed_df.pessimistic_wheelspinning]
no_indeterminate_df.rename(columns={'pessimistic_wheelspinning': 'is_wheelspinning'}, inplace=True)
no_indeterminate_df.drop(columns='optimistic_wheelspinning', inplace=True)
no_indeterminate_df

In [ ]:
print(len(processed_df))
print(len(no_indeterminate_df))

len(no_indeterminate_df.groupby(['user_id', 'skill_id']).count()) / len(processed_df.groupby(['user_id', 'skill_id']).count())

In [ ]:
print((no_indeterminate_df.user_id.nunique()))

*Your discussion about your processing decisions goes here*

## Task 2: Model Building

Train a model for your research question. 

In [ ]:
# Your code for training a model goes here
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.linear_model import LogisticRegression
# label_df = pd.DataFrame(no_indeterminate_df.groupby(['user_id','skill_id'])['is_wheelspinning'].unique())
# label_df['is_wheelspinning'] = label_df.apply(
#     lambda row: row.is_wheelspinning[0],
#     axis=1
# )
# training_df = no_indeterminate_df.drop(columns=['is_wheelspinning'])
training_df = no_indeterminate_df.copy()
datasets = {}
models = {}
results = {}

def compute_scores(clf, X_train, y_train, X_test, y_test, roundnum=3):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = balanced_accuracy_score(y_test, y_pred)

    y_pred_proba = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred_proba)
    
    return round(accuracy, roundnum), round(auc, roundnum)



for po in range(2, PO_CUTOFF+1):
    dataset = training_df[no_indeterminate_df.po < po]
    users = dataset.user_id.unique()
    users_train, users_val = train_test_split(users, test_size=0.2, random_state=0)
    
    X_train = dataset[dataset.user_id.isin(users_train)]
    X_val = dataset[dataset.user_id.isin(users_val)]
    y_train = X_train.is_wheelspinning
    X_train.drop(columns='is_wheelspinning', inplace=True)
    y_val = X_val.is_wheelspinning
    X_val.drop(columns='is_wheelspinning', inplace=True)
    datasets[po] = (X_train, X_val, y_train, y_val)
    models[po] = RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        random_state=0,
        criterion='entropy'
    )
    results[po] = compute_scores(
        models[po], X_train, y_train, X_val, y_val
    )
    print(f'For practice opportunity {po}, balanced accuracy = {results[po][0]}, AUC={results[po][1]}')

    
    



In [ ]:
import pickle
import os
import time
dir_name = time.time()
os.mkdir(f'./models/{dir_name}_{PO_CUTOFF}')
for po in range(2, PO_CUTOFF):
    os.mkdir(f'./models/{dir_name}_{PO_CUTOFF}/datasets_{po}')
    pickle.dump(datasets[po][0], open(f'./models/{dir_name}_{PO_CUTOFF}/datasets_{po}/x_train.pkl', 'wb'))
    pickle.dump(datasets[po][1], open(f'./models/{dir_name}_{PO_CUTOFF}/datasets_{po}/x_val.pkl', 'wb'))
    pickle.dump(datasets[po][2], open(f'./models/{dir_name}_{PO_CUTOFF}/datasets_{po}/y_train.pkl', 'wb'))
    pickle.dump(datasets[po][3], open(f'./models/{dir_name}_{PO_CUTOFF}/datasets_{po}/y_val.pkl', 'wb'))
    pickle.dump(models[po], open(f'./models/{dir_name}_{PO_CUTOFF}/model_{po}.pkl', 'wb'))
pickle.dump(results, open(f'./models/{dir_name}_{PO_CUTOFF}/results.pkl','wb'))

*Your discussion about your model training goes here*

## Task 3: Model Evaluation
In this task, you will use metrics to evaluate your model.

In [ ]:
# Your code for model evaluation goes here

*Your discussion/interpretation about your model's behavior goes here*

## Task 4: Team Reflection
Please describe the contributions of each team member to Milestone 4. Reflect on how you worked as team: what went well, what can be improved for the next milestone?

*Your discussion about team responsibilities goes here*